# 🧠 Media Bias Classifier

This project proposes a machine learning-based News Bias Detection System that analyses news articles to identify potential biases in reporting. By leveraging natural language processing (NLP) and explainable AI techniques, the system will assess textual content and detect linguistic patterns that indicate bias.

Bias in news media can shape public opinion and influence decision-making, making it essential to recognize and mitigate biased reporting. Traditional methods of bias detection rely on human judgment, which is subjective, time-consuming, and inconsistent. This project aims to build an automated and transparent system for detecting bias, promoting more critical and informed media consumption.

In [2]:
import nltk
import re
import string
import pandas as pd
import os
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import json
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer, TrainerCallback
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
import numpy as np
from collections import Counter
import torch
from transformers import EarlyStoppingCallback
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import re
import shap

# 📊 Preprocessing Functions

These functions clean and prepare the input for **DeBERTa**. It ensures the model focusses on meaningful, consistent, and token-friendly content.

- **🧱 Boilerplate Removal:** Removes generic news agency references (like "According to CNN" or "As reported by Reuters") to reduce noise in the input text.
- - ✅ *Why it matters:* These phrases don’t help the model learn bias—they're common across sources and can act as noise.

- **🔤 Accent Removal:** Strips accents from letters (e.g., “é” becomes “e”), improving consistency across languages. 
- - ✅ *Why it matters:* Helps reduce unnecessary vocabulary complexity for the tokenizer.

- **🧼 Preprocess_for_bert:** Applies all the text cleaning: removes boilerplate, strips accents, removes non-ASCII chars, and lowercases it.
- - ✅ *Why it matters:* Makes the text clean and simple for the tokenizer. This should be consistent throughout every text to avoid noise and bias.

- **🧼 Preprocess_for_bert_title:** Same as the function above, but without lowercasing.
- - ✅ *Why without lowercasing?* The title is shorter and often includes certain context through capitalization. For example: 

*May to lead Brexit talks*

**Original:**
Refers to Theresa May, the former UK prime minister.

**Lowercased:**
*may to lead brexit talks*
Now it could look like it's using the month of May or the modal verb “may” — completely changes meaning or adds ambiguity.

It might not look like a big deal to the human brain, but we have to be specific when training such a model.

- **✂️ truncate_for_model:** Fits both title and content into a maximum token limit, prioritizing the title. The line *remaining = max_total_tokens - len(title_tokens) - 3* accounts for 1 [CLS] token (added automatically by tokenizer) 2 [SEP] tokens (between and after title/content). Then *content_tokens = content_tokens[:max(0, remaining)]* truncates the content if it doesn't fit.
- - ✅ *Why it matters:* Transformers have a max token limit. This is making sure it never goes over that, but still prioritizes the title.

- **🎯 difficulty_score:** Simple scoring function based on total word count of title and content combined.
- - ✅ *Why it matters:* Used for curriculum sampling (easy → hard progression).

In [3]:
def remove_boilerplate(text):
    boilerplate_patterns = [
        r"(?:According to|As reported by|As per)?\s?(CNN|Fox News|Reuters|BBC|The New York Times|AP|Washington Post)[,:\s]*(reports|says|states)?",
        r"(?:Reported by|From the article in)?\s?(CNN|Fox News|Reuters|BBC|The New York Times|AP|Washington Post)"
    ]
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)
    return text.strip()

def remove_accents(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

def preprocess_for_bert(text):
    if not isinstance(text, str):
        return ""
    text = remove_boilerplate(text)
    text = remove_accents(text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text.lower().strip()
    
def truncate_for_model(content, max_total_tokens=384):
    content_tokens = tokenizer.tokenize(content)
    
    content_tokens = content_tokens[:max(0, max_total_tokens)]
    
    return tokenizer.convert_tokens_to_string(content_tokens)

def difficulty_score(title, content):
    return len(f"{title} {content}".split())

def compute_tfidf_scores(texts):
    """
    Returns average TF-IDF score for each text.
    Lower score = more ambiguous, so harder to classify.
    """
    vectorizer = TfidfVectorizer(max_features=10000)
    tfidf_matrix = vectorizer.fit_transform(texts)
    mean_scores = tfidf_matrix.mean(axis=1)
    return np.array(mean_scores).flatten()

## DeBERTa-v3

**DeBERTa-v3 (Decoding-enhanced BERT with disentangled attention)** is an advanced NLP model developed by Microsoft. It improves upon the original BERT by incorporating two key innovations: disentangled attention and enhanced decoding. Disentangled attention allows the model to better capture the relationship between words and their positions, while enhanced decoding improves the model's understanding of context. These advancements enable DeBERTa-v3 to outperform BERT and other models in many NLP tasks, providing more accurate results for complex language understanding.

### 🧠 1. Disentangled Attention: Better Word + Position Understanding
In BERT, word content and position are combined into a single embedding before being passed to the self-attention layers. This means the model might not always clearly separate what a word means from where it appears in the sentence.

DeBERTa splits the meaning of words from their positions, allowing it to analyze how word placement affects meaning more precisely.

Example:

"Only Alice punched Bob."

"Bob punched only Alice."

These sentences use the same words, but mean very different things.
DeBERTa's disentangled attention understands the difference better than BERT, which mixes meaning and position too early.

🧩 This helps the model recognize who did what, and how words like "only" change that meaning depending on position.

### 🔁 2. Enhanced Mask Decoder: Sharper Context Grasp
In BERT, the model predicts masked tokens (like [MASK]) using a simple linear layer on top of the encoder output. A simple linear layer just takes the number-y output for a word from BERT and runs it through a math formula (matrix multiplication + bias) to guess what the masked word should be. It’s like plugging the word's info into a calculator that spits out a score for every word in the dictionary, then picking the one with the highest score.

DeBERTa's decoder uses more contextual information than BERT to fill in missing or masked words during training.

Example:

"The cat sat on the [MASK]."

🔍 DeBERTa looks deeper into:

- Grammatical structure
- Likely surfaces a cat sits on
- Sentence semantics

It might confidently predict "mat", while BERT might guess less precisely (e.g., “sofa”, “floor”).

📚 This makes DeBERTa more accurate for tasks that rely on subtle language cues.

# 🔎 Analytic Approach

Detecting bias in articles might be an unclear objective, due to its abstraction. The first question to give attention to could be how exactly we detect bias, but there might be a better way to approach this. Instead, asking what bias is and how we should define it could provide more clarity.

**Multi-Class Classification** is presumably the better option. It eliminates the need for something to be classified as unbiased, as the left, center, and right all contain some form of bias.

In [4]:
# json_folder = r"C:\Fontys\Semester4\MediaBias_Predictor\Article-Bias-Prediction-main\data\jsons"

# data = []

# # Loop through JSON files
# for filename in os.listdir(json_folder):
#     if filename.endswith(".json"):
#         file_path = os.path.join(json_folder, filename)
#         with open(file_path, "r", encoding="utf-8") as f:
#             article = json.load(f)
            
#             # Extract only the fields present in the JSON
#             data.append({
#                 "ID": article.get("ID", ""),  
#                 "topic": article.get("topic", ""),  
#                 "source": article.get("source", ""),  
#                 "title": article.get("title", ""),  
#                 "date": article.get("date", ""),  
#                 "authors": article.get("authors", ""),  
#                 "content": article.get("content", ""),  
#                 "bias_text": article.get("bias_text", ""),  # Bias category (e.g., left, center, right)
#                 "url": article.get("url", ""),  
#                 "source_url": article.get("source_url", "")  
#             })

df = pd.read_csv('../news_bias_data.csv')


# df.to_csv("news_bias_data.csv", index=False)


In [5]:
df.head()

,ID,topic,source,title,date,authors,content,bias_text,url,source_url
0,004Gt3gcsotuiYmz,terrorism,New York Times - News,"Bomb Suspect Changed After Trip Abroad, Friend...",2016-09-20,N. R. Kleinfield,"Besides his most recent trip to Quetta , Mr. R...",left,http://www.nytimes.com/2016/09/20/nyregion/ahm...,www.nytimes.com
1,00eP4XD3VdMmHITE,supreme_court,Vox,Why Susan Collins claims she’s being bribed ov...,2018-09-12,"Emily Stewart, Terry Nguyen, Rebecca Jennings,...",Is Maine Republican Sen. Susan Collins being b...,left,https://www.vox.com/policy-and-politics/2018/9...,www.vox.com
2,00FTGIZEd6B8zQ4U,education,Ezra Klein,Poll: Prestigious Colleges Won't Make You Happ...,2014-05-06,Anya Kamenetz,Poll : Prestigious Colleges Wo n't Make You Ha...,left,http://www.npr.org/blogs/thetwo-way/2014/05/06...,www.npr.org
3,00HGGqBRf1kzPRlg,us_house,Breitbart News,Paul Ryan Reportedly Says No Chance for Border...,2017-09-12,Ian Mason,"House Speaker Paul Ryan , at a private dinner ...",right,http://www.breitbart.com/big-government/2017/0...,www.breitbart.com
4,00IzI5ynahBVtC9l,white_house,Guest Writer - Left,OPINION: Trump seeking change of legal fortune...,2019-07-11,Analysis Stephen Collinson,( CNN ) President Donald Trump has reason to h...,left,https://www.cnn.com/2019/07/11/politics/donald...,www.cnn.com


## 🧹 Cleaning contents and titles

In [6]:
df["cleaned_content"] = df["content"].apply(preprocess_for_bert)

df[["content", "cleaned_content"]].head()

,content,cleaned_content
0,"Besides his most recent trip to Quetta , Mr. R...","besides his most recent trip to quetta , mr. r..."
1,Is Maine Republican Sen. Susan Collins being b...,is maine republican sen. susan collins being b...
2,Poll : Prestigious Colleges Wo n't Make You Ha...,poll : prestigious colleges wo n't make you hp...
3,"House Speaker Paul Ryan , at a private dinner ...","house speaker paul ryan , at a private dinner ..."
4,( CNN ) President Donald Trump has reason to h...,() president donald trump has reason to hope h...


### ℹ️ Splitting the data

This cell below handles the data splitting to evenly distribute the left, center, and right-winged data. This is used if the model needs to be trained on less data, so that even if you take a portion of the dataset, it is still evenly distributed.

In [7]:
features = ["content", "bias_text"]

"""left_df = df[df["bias_text"] == "left"]
center_df = df[df["bias_text"] == "center"]
right_df = df[df["bias_text"] == "right"]

total_samples = 10000

# Randomly sample from each class
left_sampled = left_df.sample(n=total_samples, random_state=42)
center_sampled = center_df.sample(n=total_samples, random_state=42)
right_sampled = right_df.sample(n=total_samples, random_state=42)

df = pd.concat([left_sampled, center_sampled, right_sampled])

# Shuffle the final dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)"""

df[features].head()
# print(left_df.shape, center_df.shape, right_df.shape)
print(df.shape)


(37554, 11)


- **df.sample(frac=1, random_state=42)** means:

- - **frac=1** → take 100% of the DataFrame, but in random order.

- - **random_state=42** → makes the randomness repeatable, so every time you run it, it shuffles in the same way (helpful for debugging or reproducibility).

- **.reset_index(drop=True)**:

- - After shuffling, the original row indices are scrambled too.

- - **reset_index(drop=True)** gives the DataFrame new clean row numbers (0, 1, 2, ...) and drops the old indices.

In [8]:
# Check for missing values
print(df.isnull().sum())

# Check the distribution of bias_text labels
print(df['bias_text'].value_counts())

print("\nDataset after removing missing values:")
print(df.shape)

ID                    0
topic                 0
source                0
title                 0
date               4407
authors            9668
content               0
bias_text             0
url                   0
source_url            0
cleaned_content       0
dtype: int64
bias_text
right     13734
left      13005
center    10815
Name: count, dtype: int64

Dataset after removing missing values:
(37554, 11)


The **BERT** model requires much less data preprocessing, since it is pre-trained on **raw text**. It is in fact helpful to include the things that we otherwise exclude from other models, because it provides a certain context that BERT recognizes.

In [9]:
label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df["bias_text"]) # Representing left, center, right as 0, 1, 2

df[["bias_text", "label_id"]].head()

,bias_text,label_id
0,left,1
1,left,1
2,left,1
3,right,2
4,left,1


### ↗️ Tokenization

Before text can go into DeBERTa, it has to be converted into numbers (tokens).

- It Splits text into subwords (e.g., "unbelievable" → "un", "##believable").

- Maps words/subwords to token IDs that DeBERTa understands.

- Pads or truncates text to a fixed length (so batches are consistent).

- Returns attention masks (to tell the model which tokens are real vs. padding).

Two example sentences:

- *L, did you know* (5 tokens)
- *Gods of death love apples!* (6 tokens)

If we set the max_length to be 8, it will allow a maximum of 8 tokens. If a piece of text has more than that, it truncates it down to 8, and if it has less, it adds padding to get it to 8. In the example sentences, it would be represented as follows:

- *L, did you know [PAD] [PAD] [PAD]* (8 tokens)
- *Gods of death love apples! [PAD] [PAD]* (8 tokens)

The **attention mask** is like a signal for the model to know which tokens are real words and which ones are padding (which should be ignored). Padding is displayed as 0 and real words as 1.

The attention mask for *L, did you know [PAD] [PAD] [PAD]* would look like *[1, 1, 1, 1, 1, 0, 0, 0]*.

#### Why is this important?
Deep learning models like DeBERTa process data in batches. Having all sentences the same length allows the model to process them simultaneously. If sentences have different lengths, the model would need to deal with each sentence one by one. Computers use matrices to process data efficiently. If sentences have different lengths, you can't create a matrix because the rows (sentences) won't have the same number of columns (tokens).

# 📚 Training Pipeline

### 📊 NewsDataset class:

- __init__: Stores the tokenized input (encodings) and the labels.

- __len__: Returns the total number of samples in the dataset. Required so the model knows how many items to expect.

- __getitem__: When the model asks for a specific item (e.g., the 5th article), this method: Converts the data into PyTorch tensors and returns a dictionary with the correct format: input_ids, attention_mask, and labels

`item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}`
This line is creating a new dictionary called item. It goes through each key-value pair in self.encodings, which is a dictionary containing the tokenized input data.
`clone().detach()` is used to create a new tensor that's independent of the original one. It is like a copy to not mess with the original one.

#### Tensors

Tensors can be seen as multi-dimensional arrays:

- Scalar (0D) -> 3,14 -> torch.tensor(3.14)
- Vector (1D) -> [1, 2, 3] -> torch.tensor([1, 2, 3])
- Matrix (2D) -> [[1, 2], [3, 4]] -> torch.tensor([[1, 2], [3, 4]])
- Tensor (3D+) -> [[[...], [...]], [[...], [...]]] -> Used in images, NLP, etc.

In NLP with BERT:

A sentence becomes a 1D tensor (list of token IDs)

A batch of sentences becomes a 2D tensor (each row is a sentence)

### 📋 CurriculumTrainer class

- **get_train_dataloader**: This method is responsible for returning the dataloader used during training. Normally, the dataloader would shuffle training data, but here we turn that off to preserve the "easy-to-hard" order required for curriculum learning.

- **collate_fn**: The collate_fn is a function used to combine or “batch” data into a single batch. It is needed when you have different types of data, or the data isn't structured in a way that can be directly fed into the model. It makes sure that each input in the batch is the same size. The tokenizer does this too by adding padding to the text. The collate_fn function does the same, but provides more control.

- **drop_last=False**: When dividing data into batches, sometimes you end up with a smaller batch at the end because the total data isn't a perfect multiple of the batch size. This setting says makes sure to keep that batch. Even though it's smaller, it still gets fed into the model.

### 🤔 StratifiedKFold
**K-Fold Cross-Validation** is a technique used in machine learning to evaluate the performance of a model. The dataset is divided into **k** equal-sized subsets or "folds". The model is trained on **k-1** folds and tested on the remaining fold. This is repeated **k** times, with each fold used exactly once for testing and the remaining ones for training. **Stratified K-Fold** ensures each fold has a similar distribution of the target labels, making it more reliable for evaluating models on imbalanced datasets. The dataset used already has evenly distributed labels, but it can still act as a safety net.

### 👨‍🎓 Curriculum Learning

Instead of feeding the model data randomly, curriculum learning starts with easier examples first, and moves on to harder ones later, because the model starts off being "stupid".

##### 🔎 Line by line breakdown

`train_data = [(titles[i], df["content"].iloc[i], labels[i]) for i in train_index]` takes the training indices from the split. For each index it grabs the:
- title
- content
- label

It creates a list of tuples like:
`[("Why something is great", "Something is amazing, because...", 0), (...), ...]`

`train_titles, train_contents, train_labels = zip(*train_data)` creates separate lists for all the titles, contents, and labels.

`train_texts_full = [f"{title} {content}" for title, content in zip(train_titles, train_contents)]` combines the title and content into one text.

`tfidf_scores = compute_tfidf_scores(train_texts_full)` High TF-IDF score → article is unique, has rare words. Low score → article is generic, uses common words.

`train_data_with_scores = list(zip(train_titles, train_contents, train_labels, tfidf_scores))` now each sample has a score.

`train_data_with_scores.sort(key=lambda x: x[3], reverse=True)` sorts the data using the fourth element of the tuple, which is the **tfidf_score**. `reverse=True` determines that the order should be descending.

`train_titles, train_contents, train_labels, _ = zip(*train_data_with_scores)` now the data is sorted using the tfidf scores, so they can be ignored and the data can now be used for curriculum learning.

In [11]:
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import StratifiedKFold as skf
from collections import Counter
import torch
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base", num_labels=3)

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

class CurriculumTrainer(Trainer):
    def get_train_dataloader(self):
        # Disable shuffling for curriculum learning
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            shuffle=False,
            collate_fn=self.data_collator,
            drop_last=False,
        )

contents = df["cleaned_content"].tolist()
labels = df["label_id"].tolist()
accuracies = []  # Initialize accuracies list for storing fold results

stratified_kfold = skf(n_splits=3, shuffle=True, random_state=42)

model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=3)

for fold, (train_index, val_index) in enumerate(stratified_kfold.split(contents, labels)):
    print(f"\n===== Fold {fold + 1} =====")

    # STEP 1: Grab training data using the indices from StratifiedKFold
    train_data = [
        (df["cleaned_content"].iloc[i], labels[i]) for i in train_index
    ]

    # STEP 2: Unpack (unzip) train_data into individual lists
    train_contents, train_labels = zip(*train_data)

    # STEP 3: Compute TF-IDF scores for all the combined texts
    tfidf_scores = compute_tfidf_scores(train_contents)

    # STEP 4: Attach the scores to the data for sorting
    train_data_with_scores = list(zip(train_contents, train_labels, tfidf_scores))

    # STEP 5: Sort the data — lowest TF-IDF (most ambiguous) comes last in curriculum
    train_data_with_scores.sort(key=lambda x: x[2], reverse=True)

    # STEP 6: Unpack the sorted data (ignore the scores now)
    train_contents, train_labels, _ = zip(*train_data_with_scores)

    val_contents = [df["cleaned_content"].iloc[i] for i in val_index]
    val_labels = [labels[i] for i in val_index]

    print(f"Training label distribution: {Counter(train_labels)}")
    print(f"Validation label distribution: {Counter(val_labels)}")

    train_texts = [truncate_for_model(c) for c in train_contents]
    val_texts = [truncate_for_model(c) for c in val_contents]

    train_encodings = tokenizer(train_texts, max_length=384, truncation=True, padding=True, return_tensors='pt')
    val_encodings = tokenizer(val_texts, max_length=384, truncation=True, padding=True, return_tensors='pt')

    train_dataset = NewsDataset(train_encodings, train_labels)
    val_dataset = NewsDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold}",
        num_train_epochs=6,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir=None,
        logging_steps=999999,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        save_total_limit=1,
        report_to="none",
        learning_rate=3e-5,
        fp16=True,
        lr_scheduler_type='cosine',
        gradient_accumulation_steps=2,
    )

    def compute_accuracy(pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis=1)
        accuracy = (preds == labels).mean()
        return {"accuracy": accuracy}

    trainer = CurriculumTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_accuracy
    )

    trainer.train()

    eval_result = trainer.evaluate()
    trainer.save_model(f"saved_models/fold_{fold}")
    tokenizer.save_pretrained(f"saved_models/fold_{fold}")
    print(f"Fold {fold + 1} Accuracy: {eval_result['eval_accuracy']:.4f}")
    accuracies.append(eval_result['eval_accuracy'])

    predictions = trainer.predict(val_dataset)
    pred_labels = np.argmax(predictions.predictions, axis=1)
    print("\nClassification Report:")
    print(classification_report(val_labels, pred_labels))

print("\n===== Cross-Validation Complete =====")
print(f"Mean Accuracy: {np.mean(accuracies):.4f}")

C:\Users\wtert\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Fold 1 =====
Training label distribution: Counter({2: 9156, 1: 8670, 0: 7210})
Validation label distribution: Counter({2: 4578, 1: 4335, 0: 3605})


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.521346,0.791820
2,No log,0.661159,0.778719
3,No log,0.654579,0.811392
4,No log,0.748139,0.820578
5,No log,0.770841,0.841109
6,No log,0.784108,0.841109


Fold 1 Accuracy: 0.8411

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.83      0.82      3605
           1       0.84      0.86      0.85      4335
           2       0.86      0.84      0.85      4578

    accuracy                           0.84     12518
   macro avg       0.84      0.84      0.84     12518
weighted avg       0.84      0.84      0.84     12518


===== Fold 2 =====
Training label distribution: Counter({2: 9156, 1: 8670, 0: 7210})
Validation label distribution: Counter({2: 4578, 1: 4335, 0: 3605})


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.244839,0.908771
2,No log,0.242425,0.919636
3,No log,0.272056,0.930420
4,No log,0.350709,0.924589
5,No log,0.301155,0.941125
6,No log,0.263759,0.945838


Fold 2 Accuracy: 0.9458

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      3605
           1       0.94      0.96      0.95      4335
           2       0.96      0.95      0.96      4578

    accuracy                           0.95     12518
   macro avg       0.94      0.94      0.94     12518
weighted avg       0.95      0.95      0.95     12518


===== Fold 3 =====
Training label distribution: Counter({2: 9156, 1: 8670, 0: 7210})
Validation label distribution: Counter({2: 4578, 1: 4335, 0: 3605})


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.093836,0.968046
2,No log,0.146159,0.957741
3,No log,0.140817,0.969564
4,No log,0.081093,0.983783
5,No log,0.064833,0.987458
6,No log,0.059774,0.988736


Fold 3 Accuracy: 0.9887

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3605
           1       0.99      0.99      0.99      4335
           2       0.99      0.99      0.99      4578

    accuracy                           0.99     12518
   macro avg       0.99      0.99      0.99     12518
weighted avg       0.99      0.99      0.99     12518


===== Cross-Validation Complete =====
Mean Accuracy: 0.9252


# 🔎 Evaluation

In [5]:
from sklearn.metrics import ConfusionMatrixDisplay

# Generate the confusion matrix
cm = confusion_matrix(val_labels, pred_labels)

# Normalize the confusion matrix to show percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Display the confusion matrix with percentages
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percentage, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix (Percentages)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

NameError: name 'val_labels' is not defined